In [22]:
import sys
import os
from scipy.optimize import curve_fit

# Get the project root (one level above notebooks/)
project_root = os.path.abspath("..")
sys.path.append(project_root)

print("Added to Python path:", project_root)


Added to Python path: c:\Users\YooNi\OneDrive\Desktop\Majorana-Neutrino-Hunt


In [23]:
import h5py
import numpy as np
import pandas as pd
# from src.parameters.tail_features import compute_LQ80

In [24]:
train_file = "../data/old/MJD_Train_0.hdf5"

with h5py.File(train_file, "r") as f:
    waveforms = np.array(f["raw_waveform"])
    ids = np.array(f["id"])

print("Loaded", len(waveforms), "waveforms")


Loaded 65000 waveforms


In [25]:
def print_hdf5_structure(name, obj):
    print(name)

with h5py.File(train_file, "r") as f:
    f.visititems(print_hdf5_structure)


detector
energy_label
id
psd_label_dcr
psd_label_high_avse
psd_label_low_avse
psd_label_lq
raw_waveform
run_number
tp0


In [26]:
def estimate_baseline(y, n_samples=200):
    """
    Returns baseline (mean, std) from the first n_samples.
    """
    y0 = np.asarray(y, dtype=float)[:n_samples]
    return float(np.mean(y0)), float(np.std(y0))


In [27]:
def exponential(t, a, tau1, b, tau2):
    """
    Double exponential decay model of the form:
        a * exp(-t/tau1) + b * exp(-t/tau2)

    This models the long falling edge of HPGe waveforms.
    """
    return a * np.exp(-t / tau1) + b * np.exp(-t / tau2)


In [28]:
#pole zero correction
def pole_zero_correction(waveform, use_pz=True, min_tail_len=10):
    """
    Applies pole-zero correction to a given waveform.

    Returns:
        waveform_pz (np.array): The PZ-corrected full waveform.
        waveform_tail_corrected (np.array): The PZ-corrected tail portion of the waveform (from t98 onward).
    """
    waveform = np.asarray(waveform, dtype=float)
    if not use_pz:
        return waveform.copy(), waveform.copy()

    peak_value = float(np.max(waveform))
    if (not np.isfinite(peak_value)) or peak_value <= 0:
        return waveform.copy(), waveform.copy()

    # Isolate the tail (starting at 98% of the peak)
    idx98 = np.where(waveform >= 0.98 * peak_value)[0]
    if len(idx98) == 0:
        return waveform.copy(), waveform.copy()
    t98 = int(idx98[0])

    tail_values = waveform[t98:]
    tail_time = np.arange(tail_values.size, dtype=float)

    # If the tail is too short, we cannot fit 4 parameters
    if tail_values.size < min_tail_len:
        return waveform.copy(), tail_values.copy()

    # Initial guess
    p0 = [tail_values[0], 300.0, tail_values[0] * 0.1, 1500.0]

    try:
        params, _ = curve_fit(
            exponential,
            tail_time,
            tail_values,
            p0=p0,
            maxfev=20000
        )
    except (RuntimeError, ValueError, TypeError):
        return waveform.copy(), tail_values.copy()

    # Calculate the correction factor and apply it
    f_decay = exponential(tail_time, *params)

    # Estimate the initial value of the tail from a few samples near t98
    end = min(t98 + 5, len(waveform))
    f_t0 = float(np.mean(waveform[t98:end]))

    eps = 1e-12
    f_pz = f_t0 / (f_decay + eps)

    waveform_tail_corrected = tail_values * f_pz
    waveform_pz = waveform.copy()
    waveform_pz[t98:] = waveform_tail_corrected

    return waveform_pz, waveform_tail_corrected


In [29]:
def compute_LQ80(waveform):
    """
    Late Charge 80:
    Area difference between raw and PZ-corrected waveform
    starting at the 80 percent rising edge.
    """
  
    waveform_pz, _ = pole_zero_correction(waveform, use_pz=True)

    y  = np.asarray(waveform, dtype=float)
    yc = np.asarray(waveform_pz, dtype=float)

    # Baseline
    baseline, _ = estimate_baseline(y)

    # Peak
    peak_val = float(np.max(y))
    target = baseline + 0.80 * (peak_val - baseline)

    # Rising-edge crossing
    idx = np.where(y >= target)[0]
    if len(idx) == 0:
        return np.nan

    i80 = int(idx[0])

    # Time index for integration
    t = np.arange(len(y), dtype=float)

    area_raw  = float(np.trapezoid(y[i80:],  t[i80:]))
    area_corr = float(np.trapezoid(yc[i80:], t[i80:]))

    return area_raw - area_corr


In [30]:
OUTPUT_DIR = "finalcsveunice"
os.makedirs(OUTPUT_DIR, exist_ok=True)


In [10]:
all_ids = []
all_LQ80 = []

for train_idx in range(16):
    train_file = f"../data/old/MJD_Train_{train_idx}.hdf5"
    if not os.path.exists(train_file):
        print(f"Skipping missing file: {train_file}")
        continue

    print(f"\nLoading {train_file}")

    with h5py.File(train_file, "r") as f:
        waveforms = np.array(f["raw_waveform"])
        ids = np.array(f["id"])

    print(f"  Waveforms: {len(waveforms)}")

    for i, wf in enumerate(waveforms):
        if i % 5000 == 0:
            print(f"    LQ80 Train_{train_idx} {i}/{len(waveforms)}")

        all_LQ80.append(compute_LQ80(wf))
        all_ids.append(f"{ids[i]}_train_{train_idx}")

all_LQ80 = np.array(all_LQ80, dtype=float)
all_LQ80[~np.isfinite(all_LQ80)] = np.nan

df_lq80 = pd.DataFrame({
    "id": all_ids,
    "LQ80": all_LQ80
})

output_path_lq80 = os.path.join(OUTPUT_DIR, "LQ80_train_all.csv")
df_lq80.to_csv(output_path_lq80, index=False)

print("\nSaved combined LQ80 CSV to:", output_path_lq80)
print(df_lq80.head())
print(df_lq80["LQ80"].describe())
print("NaNs:", df_lq80["LQ80"].isna().sum())



Loading ../data/old/MJD_Train_0.hdf5
  Waveforms: 65000
    LQ80 Train_0 0/65000


C:\Users\YooNi\AppData\Local\Temp\ipykernel_16004\1352040052.py:8: RuntimeWarning: overflow encountered in exp
  return a * np.exp(-t / tau1) + b * np.exp(-t / tau2)
C:\Users\YooNi\AppData\Local\Temp\ipykernel_16004\1352040052.py:8: RuntimeWarning: overflow encountered in multiply
  return a * np.exp(-t / tau1) + b * np.exp(-t / tau2)


    LQ80 Train_0 5000/65000
    LQ80 Train_0 10000/65000
    LQ80 Train_0 15000/65000
    LQ80 Train_0 20000/65000
    LQ80 Train_0 25000/65000


C:\Users\YooNi\AppData\Local\Temp\ipykernel_16004\559366876.py:35: OptimizeWarning: Covariance of the parameters could not be estimated
  params, _ = curve_fit(


    LQ80 Train_0 30000/65000
    LQ80 Train_0 35000/65000
    LQ80 Train_0 40000/65000
    LQ80 Train_0 45000/65000
    LQ80 Train_0 50000/65000
    LQ80 Train_0 55000/65000
    LQ80 Train_0 60000/65000

Loading ../data/old/MJD_Train_1.hdf5
  Waveforms: 65000
    LQ80 Train_1 0/65000
    LQ80 Train_1 5000/65000
    LQ80 Train_1 10000/65000
    LQ80 Train_1 15000/65000
    LQ80 Train_1 20000/65000
    LQ80 Train_1 25000/65000
    LQ80 Train_1 30000/65000
    LQ80 Train_1 35000/65000
    LQ80 Train_1 40000/65000
    LQ80 Train_1 45000/65000
    LQ80 Train_1 50000/65000
    LQ80 Train_1 55000/65000
    LQ80 Train_1 60000/65000

Loading ../data/old/MJD_Train_2.hdf5
  Waveforms: 65000
    LQ80 Train_2 0/65000
    LQ80 Train_2 5000/65000
    LQ80 Train_2 10000/65000
    LQ80 Train_2 15000/65000
    LQ80 Train_2 20000/65000
    LQ80 Train_2 25000/65000
    LQ80 Train_2 30000/65000
    LQ80 Train_2 35000/65000
    LQ80 Train_2 40000/65000
    LQ80 Train_2 45000/65000
    LQ80 Train_2 50000/650

In [31]:
# LQ80 Test
all_test_ids = []
all_LQ80_test = []

for test_idx in range(6):
    test_file = f"../data/old/MJD_Test_{test_idx}.hdf5"
    if not os.path.exists(test_file):
        print(f"Skipping missing file: {test_file}")
        continue

    print(f"\nLoading {test_file}")

    with h5py.File(test_file, "r") as f:
        waveforms_test = np.array(f["raw_waveform"])
        ids_test = np.array(f["id"])

    print(f"  Waveforms: {len(waveforms_test)}")

    for i, wf in enumerate(waveforms_test):
        if i % 5000 == 0:
            print(f"    LQ80 Test_{test_idx} {i}/{len(waveforms_test)}")

        all_LQ80_test.append(compute_LQ80(wf))
        all_test_ids.append(f"{ids_test[i]}_test_{test_idx}")

all_LQ80_test = np.array(all_LQ80_test, dtype=float)
all_LQ80_test[~np.isfinite(all_LQ80_test)] = np.nan

df_lq80_test = pd.DataFrame({
    "id": all_test_ids,
    "LQ80": all_LQ80_test
})

output_path_lq80_test = os.path.join(OUTPUT_DIR, "LQ80_test_all.csv")
df_lq80_test.to_csv(output_path_lq80_test, index=False)

print("\nSaved combined LQ80 TEST CSV to:", output_path_lq80_test)
print(df_lq80_test.head())
print(df_lq80_test["LQ80"].describe())
print("NaNs:", df_lq80_test["LQ80"].isna().sum())



Loading ../data/old/MJD_Test_0.hdf5
  Waveforms: 65000
    LQ80 Test_0 0/65000


C:\Users\YooNi\AppData\Local\Temp\ipykernel_25292\1352040052.py:8: RuntimeWarning: overflow encountered in exp
  return a * np.exp(-t / tau1) + b * np.exp(-t / tau2)
C:\Users\YooNi\AppData\Local\Temp\ipykernel_25292\1352040052.py:8: RuntimeWarning: overflow encountered in multiply
  return a * np.exp(-t / tau1) + b * np.exp(-t / tau2)


    LQ80 Test_0 5000/65000
    LQ80 Test_0 10000/65000
    LQ80 Test_0 15000/65000
    LQ80 Test_0 20000/65000
    LQ80 Test_0 25000/65000
    LQ80 Test_0 30000/65000
    LQ80 Test_0 35000/65000


C:\Users\YooNi\AppData\Local\Temp\ipykernel_25292\559366876.py:35: OptimizeWarning: Covariance of the parameters could not be estimated
  params, _ = curve_fit(


    LQ80 Test_0 40000/65000
    LQ80 Test_0 45000/65000
    LQ80 Test_0 50000/65000
    LQ80 Test_0 55000/65000
    LQ80 Test_0 60000/65000

Loading ../data/old/MJD_Test_1.hdf5
  Waveforms: 65000
    LQ80 Test_1 0/65000
    LQ80 Test_1 5000/65000
    LQ80 Test_1 10000/65000
    LQ80 Test_1 15000/65000
    LQ80 Test_1 20000/65000
    LQ80 Test_1 25000/65000
    LQ80 Test_1 30000/65000
    LQ80 Test_1 35000/65000
    LQ80 Test_1 40000/65000
    LQ80 Test_1 45000/65000
    LQ80 Test_1 50000/65000
    LQ80 Test_1 55000/65000
    LQ80 Test_1 60000/65000

Loading ../data/old/MJD_Test_2.hdf5
  Waveforms: 65000
    LQ80 Test_2 0/65000
    LQ80 Test_2 5000/65000
    LQ80 Test_2 10000/65000
    LQ80 Test_2 15000/65000
    LQ80 Test_2 20000/65000
    LQ80 Test_2 25000/65000
    LQ80 Test_2 30000/65000
    LQ80 Test_2 35000/65000
    LQ80 Test_2 40000/65000
    LQ80 Test_2 45000/65000
    LQ80 Test_2 50000/65000
    LQ80 Test_2 55000/65000
    LQ80 Test_2 60000/65000

Loading ../data/old/MJD_Test_